<a href="https://colab.research.google.com/github/bathaee/QGAN-EQ-GAN/blob/main/Quantum_Error_Mitigation_With_Qiskit_And_Mitiq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install qiskit --upgrade
!pip install qiskit-machine-learning
#!pip install qiskit[nature]>=0.27
!pip install qiskit-optimization[cplex]
!pip install mitiq
#!pip install amazon-braket-sdk
#!pip install pyquil
#!pip install cirq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
# 1. Define a quantum circuit
import numpy as np
from qiskit import QuantumCircuit

def get_circuit():
    qc = QuantumCircuit(2)

    # CDR works better if the circuit is not too short. So we increase its depth.
    for i in range(5): 
        qc.h(0) # Clifford
        qc.h(1) # Clifford
        qc.rz(1.75, 0)
        qc.rz(2.31, 1)
        qc.cx(0,1) # Clifford
        qc.rz(-1.17, 1)
        qc.rz(3.23, 0)
        qc.rx(np.pi/2, 0) # Clifford
        qc.rx(np.pi/2, 1) # Clifford

    # We need to measure the qubits
    qc.measure_all()
    return qc

In [43]:
circ = get_circuit()
print(circ)

        ┌───┐┌──────────┐      ┌──────────┐┌─────────┐┌───┐┌──────────┐     »
   q_0: ┤ H ├┤ Rz(1.75) ├──■───┤ Rz(3.23) ├┤ Rx(π/2) ├┤ H ├┤ Rz(1.75) ├──■──»
        ├───┤├──────────┤┌─┴─┐┌┴──────────┤├─────────┤├───┤├──────────┤┌─┴─┐»
   q_1: ┤ H ├┤ Rz(2.31) ├┤ X ├┤ Rz(-1.17) ├┤ Rx(π/2) ├┤ H ├┤ Rz(2.31) ├┤ X ├»
        └───┘└──────────┘└───┘└───────────┘└─────────┘└───┘└──────────┘└───┘»
meas: 2/════════════════════════════════════════════════════════════════════»
                                                                            »
«         ┌──────────┐┌─────────┐┌───┐┌──────────┐      ┌──────────┐┌─────────┐»
«   q_0: ─┤ Rz(3.23) ├┤ Rx(π/2) ├┤ H ├┤ Rz(1.75) ├──■───┤ Rz(3.23) ├┤ Rx(π/2) ├»
«        ┌┴──────────┤├─────────┤├───┤├──────────┤┌─┴─┐┌┴──────────┤├─────────┤»
«   q_1: ┤ Rz(-1.17) ├┤ Rx(π/2) ├┤ H ├┤ Rz(2.31) ├┤ X ├┤ Rz(-1.17) ├┤ Rx(π/2) ├»
«        └───────────┘└─────────┘└───┘└──────────┘└───┘└───────────┘└─────────┘»
«meas: 2/════════════════════════════════════════

In [90]:
from mitiq import Observable, PauliString

obs = Observable(PauliString("ZZ"), PauliString("X", coeff=-1.75))
print(obs)
type(obs)

Z(q(0))*Z(q(1)) + (-1.75+0j)*X(q(0))


mitiq.observable.observable.Observable

In [81]:
# 2. Define an executor
from mitiq.interface.mitiq_qiskit import qiskit_utils
from qiskit import QuantumCircuit, execute, Aer
import qiskit_aer.noise as noise

# Error probabilities
prob_1 = 0.005  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

def sim_noise(qc):
    return qiskit_utils.execute_with_shots_and_noise(qc, obs.matrix(), noise_model, 4096)


In [97]:
def sim(qc):
    return qiskit_utils.execute_with_shots(qc, obs.matrix(), 4096)

In [83]:
type(sim)

function

In [96]:
ideal_measurement = obs.expectation( get_circuit(),sim).real
print("ideal_measurement = ",ideal_measurement)

ValueError: ignored

In [92]:
unmitigated_measurement = obs.expectation(get_circuit(), sim_noise).real
print("unmitigated_measurement = ", unmitigated_measurement)

ValueError: ignored

In [94]:
from mitiq import cdr

mitigated_measurement = cdr.execute_with_cdr(
    get_circuit(),
    sim_noise,
    observable=obs.matrix(),
    simulator=sim,
    seed=0).real
print("mitigated_measurement = ", mitigated_measurement)

ValueError: ignored